In [1]:
import pandas as pd
import numpy as np

In [2]:
df_sales = pd.read_csv("sales_train_validation.csv")
df_prices = pd.read_csv("sell_prices.csv")
df_calendar = pd.read_csv("calendar.csv")

In [13]:
pd.set_option('display.max_columns', 7000)

In [3]:
stores = df_sales.store_id.unique()

In [4]:
def generate_store_df(store_cod):
    df_sales_AUX = df_sales[df_sales["store_id"] == store_cod].copy()
    df_prices_AUX = df_prices[df_prices["store_id"] == store_cod].copy()
    df_calendar_AUX = df_calendar.copy()
    
    df_sales_AUX.drop(["id", "dept_id", "cat_id", "store_id", "state_id"], axis=1, inplace=True)
    
    keep_calendar_cols = ["wm_yr_wk","date", "d", "event_name_1", "event_type_1", "event_name_2", "event_type_2"]
    # check  the state the store is located to select the right snap column
    if "CA" in store_cod:
        keep_calendar_cols.append("snap_CA")
    elif "TX" in store_cod:
        keep_calendar_cols.append("snap_TX")
    elif "WI" in store_cod:
        keep_calendar_cols.append("snap_WI")
    
    df_calendar_AUX = df_calendar_AUX[keep_calendar_cols]
    
    df_prices_AUX = df_prices_AUX.pivot(index="wm_yr_wk", columns="item_id", values="sell_price")
    df_prices_AUX.fillna(method="backfill", axis=0, inplace=True)
    
    df_calendar_AUX = df_calendar_AUX.merge(df_prices_AUX, on="wm_yr_wk", how="left")
    df_calendar_AUX.drop("wm_yr_wk", axis=1, inplace=True)
    
    sales_days = ["d_"+str(x) for x in range(1, 1914)]
    df_sales_AUX = pd.melt(df_sales_AUX, id_vars=["item_id"], value_vars=sales_days)
    df_sales_AUX.rename(columns = {'variable':'d'}, inplace=True)
    df_sales_AUX = df_sales_AUX.pivot(index='d', columns='item_id', values='value')
    df_sales_AUX.reset_index(inplace=True)
        
    result = df_sales_AUX.merge(df_calendar_AUX, on="d", how="left")
    result.rename(columns = {'date_y':'date'}, inplace=True)
    result["d"] = result["d"].apply(lambda x : int(x.split("_")[1]))
    result.set_index("d", drop=True, inplace=True)
    result.sort_index(inplace=True)
    result.columns = result.columns.str.replace("_x", "_sells")
    result.columns = result.columns.str.replace("_y", "_price")
    
    return result

In [5]:
datasets = dict()
for s in stores:
    print(s + "\n")
    aux = generate_store_df(s)
    datasets[s] = aux

CA_1

CA_2

CA_3

CA_4

TX_1

TX_2

TX_3

WI_1

WI_2

WI_3



In [10]:
datasets["CA_3"]

,FOODS_1_001_sells,FOODS_1_002_sells,FOODS_1_003_sells,FOODS_1_004_sells,FOODS_1_005_sells,FOODS_1_006_sells,FOODS_1_008_sells,FOODS_1_009_sells,FOODS_1_010_sells,FOODS_1_011_sells,...,HOUSEHOLD_2_507_price,HOUSEHOLD_2_508_price,HOUSEHOLD_2_509_price,HOUSEHOLD_2_510_price,HOUSEHOLD_2_511_price,HOUSEHOLD_2_512_price,HOUSEHOLD_2_513_price,HOUSEHOLD_2_514_price,HOUSEHOLD_2_515_price,HOUSEHOLD_2_516_price
d,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,0,10,0,0,0,0,1,...,6.97,14.87,8.97,4.27,2.87,3.97,2.78,18.97,1.97,5.94
2,2,0,1,0,3,0,0,0,0,0,...,6.97,14.87,8.97,4.27,2.87,3.97,2.78,18.97,1.97,5.94
3,1,1,1,0,2,0,0,0,0,2,...,6.97,14.87,8.97,4.27,2.87,3.97,2.78,18.97,1.97,5.94
4,1,2,3,0,10,0,0,0,0,0,...,6.97,14.87,8.97,4.27,2.87,3.97,2.78,18.97,1.97,5.94
5,1,0,0,0,1,1,0,0,0,0,...,6.97,14.87,8.97,4.27,2.87,3.97,2.78,18.97,1.97,5.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909,0,1,3,0,1,1,0,0,2,0,...,6.97,15.94,8.97,3.93,2.74,3.97,2.78,19.54,1.97,5.94
1910,0,0,0,0,0,0,0,0,0,1,...,6.97,15.94,8.97,3.93,2.74,3.97,2.78,19.54,1.97,5.94
1911,0,0,0,0,3,0,0,0,0,0,...,6.97,15.94,8.97,3.93,2.74,3.97,2.78,19.54,1.97,5.94


In [14]:
datasets["CA_3"].head()

,FOODS_1_001_sells,FOODS_1_002_sells,FOODS_1_003_sells,FOODS_1_004_sells,FOODS_1_005_sells,FOODS_1_006_sells,FOODS_1_008_sells,FOODS_1_009_sells,FOODS_1_010_sells,FOODS_1_011_sells,FOODS_1_012_sells,FOODS_1_013_sells,FOODS_1_014_sells,FOODS_1_015_sells,FOODS_1_016_sells,FOODS_1_017_sells,FOODS_1_018_sells,FOODS_1_019_sells,FOODS_1_020_sells,FOODS_1_021_sells,FOODS_1_022_sells,FOODS_1_023_sells,FOODS_1_024_sells,FOODS_1_025_sells,FOODS_1_026_sells,FOODS_1_027_sells,FOODS_1_028_sells,FOODS_1_029_sells,FOODS_1_030_sells,FOODS_1_031_sells,FOODS_1_032_sells,FOODS_1_033_sells,FOODS_1_034_sells,FOODS_1_035_sells,FOODS_1_036_sells,FOODS_1_037_sells,FOODS_1_038_sells,FOODS_1_039_sells,FOODS_1_040_sells,FOODS_1_041_sells,FOODS_1_042_sells,FOODS_1_043_sells,FOODS_1_044_sells,FOODS_1_045_sells,FOODS_1_046_sells,FOODS_1_047_sells,FOODS_1_048_sells,FOODS_1_049_sells,FOODS_1_050_sells,FOODS_1_051_sells,FOODS_1_052_sells,FOODS_1_053_sells,FOODS_1_054_sells,FOODS_1_055_sells,FOODS_1_056_sells,FOODS_1_057_sells,FOODS_1_058_sells,FOODS_1_059_sells,FOODS_1_060_sells,FOODS_1_061_sells,FOODS_1_062_sells,FOODS_1_063_sells,FOODS_1_064_sells,FOODS_1_065_sells,FOODS_1_066_sells,FOODS_1_067_sells,FOODS_1_068_sells,FOODS_1_069_sells,FOODS_1_070_sells,FOODS_1_071_sells,FOODS_1_072_sells,FOODS_1_073_sells,FOODS_1_074_sells,FOODS_1_075_sells,FOODS_1_076_sells,FOODS_1_077_sells,FOODS_1_078_sells,FOODS_1_079_sells,FOODS_1_080_sells,FOODS_1_081_sells,FOODS_1_082_sells,FOODS_1_083_sells,FOODS_1_084_sells,FOODS_1_085_sells,FOODS_1_086_sells,FOODS_1_087_sells,FOODS_1_088_sells,FOODS_1_089_sells,FOODS_1_090_sells,FOODS_1_091_sells,FOODS_1_092_sells,FOODS_1_093_sells,FOODS_1_094_sells,FOODS_1_095_sells,FOODS_1_096_sells,FOODS_1_097_sells,FOODS_1_098_sells,FOODS_1_099_sells,FOODS_1_101_sells,FOODS_1_102_sells,FOODS_1_103_sells,FOODS_1_104_sells,FOODS_1_105_sells,FOODS_1_106_sells,FOODS_1_107_sells,FOODS_1_108_sells,FOODS_1_109_sells,FOODS_1_110_sells,FOODS_1_111_sells,FOODS_1_112_sells,FOODS_1_113_sells,FOODS_1_114_sells,FOODS_1_115_sells,FOODS_1_116_sells,FOODS_1_117_sells,FOODS_1_118_sells,FOODS_1_119_sells,FOODS_1_120_sells,FOODS_1_121_sells,FOODS_1_122_sells,FOODS_1_123_sells,FOODS_1_124_sells,FOODS_1_125_sells,FOODS_1_126_sells,FOODS_1_127_sells,FOODS_1_128_sells,FOODS_1_129_sells,FOODS_1_130_sells,FOODS_1_131_sells,FOODS_1_132_sells,FOODS_1_133_sells,FOODS_1_134_sells,FOODS_1_135_sells,FOODS_1_136_sells,FOODS_1_137_sells,FOODS_1_138_sells,FOODS_1_139_sells,FOODS_1_140_sells,FOODS_1_141_sells,FOODS_1_142_sells,FOODS_1_143_sells,FOODS_1_144_sells,FOODS_1_145_sells,FOODS_1_146_sells,FOODS_1_147_sells,FOODS_1_148_sells,FOODS_1_149_sells,FOODS_1_150_sells,FOODS_1_151_sells,FOODS_1_152_sells,FOODS_1_153_sells,FOODS_1_154_sells,FOODS_1_155_sells,FOODS_1_156_sells,FOODS_1_157_sells,FOODS_1_158_sells,FOODS_1_159_sells,FOODS_1_160_sells,FOODS_1_161_sells,FOODS_1_162_sells,FOODS_1_163_sells,FOODS_1_164_sells,FOODS_1_166_sells,FOODS_1_167_sells,FOODS_1_168_sells,FOODS_1_169_sells,FOODS_1_170_sells,FOODS_1_171_sells,FOODS_1_172_sells,FOODS_1_173_sells,FOODS_1_174_sells,FOODS_1_175_sells,FOODS_1_176_sells,FOODS_1_177_sells,FOODS_1_178_sells,FOODS_1_179_sells,FOODS_1_180_sells,FOODS_1_181_sells,FOODS_1_182_sells,FOODS_1_183_sells,FOODS_1_184_sells,FOODS_1_185_sells,FOODS_1_186_sells,FOODS_1_187_sells,FOODS_1_188_sells,FOODS_1_189_sells,FOODS_1_190_sells,FOODS_1_191_sells,FOODS_1_192_sells,FOODS_1_193_sells,FOODS_1_194_sells,FOODS_1_195_sells,FOODS_1_196_sells,FOODS_1_197_sells,FOODS_1_198_sells,FOODS_1_199_sells,FOODS_1_200_sells,FOODS_1_201_sells,FOODS_1_202_sells,FOODS_1_203_sells,FOODS_1_204_sells,FOODS_1_205_sells,FOODS_1_206_sells,FOODS_1_207_sells,FOODS_1_208_sells,FOODS_1_209_sells,FOODS_1_210_sells,FOODS_1_211_sells,FOODS_1_212_sells,FOODS_1_213_sells,FOODS_1_214_sells,FOODS_1_215_sells,FOODS_1_216_sells,FOODS_1_217_sells,FOODS_1_218_sells,FOODS_1_219_sells,FOODS_2_001_sells,FOODS_2_002_sells,FOODS_2_003_sells,FOODS_2_004_sells,FOODS_2_005_sells,FOODS_2_006_sells,FOO